# Page 1-3

In [1]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:124.0) Gecko/20100101 Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; rv:123.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (X11; openSUSE; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.140 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.139 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.170 Safari/537.36",
    "Mozilla/5.0 (X11; Linux Mint x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Manjaro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.193 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.151 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Kali) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.145 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Solus) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.95 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.148 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.120 Safari/537.36"
]



random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/moisturizers-skin-care-beauty/-/N-5xtzf?moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list = []

def scrape_products():
    global product_list, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Moisturizer",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：74 条


In [2]:
df = pd.DataFrame(product_list)
df

product_name            brand  \
0      Olay Super Facial Moisturizer - SPF 30 - 1.7oz             Olay   
1   La Roche Posay Toleriane Double Repair Face Mo...   La Roche Posay   
2   good.clean.goop beauty The Nutrient-Rich Daily...  good.clean.goop   
3   e.l.f. Holy Hydration Face Cream Fragrance Fre...           e.l.f.   
4   L'Oreal Paris Age Perfect Cell Renewal Midnigh...    L'Oreal Paris   
..                                                ...              ...   
69     RoC Derm Correxion Dual Eye Cream - 0.68 fl oz              RoC   
70  Cetaphil Rich Hydrating Face Cream with Hyalur...         Cetaphil   
71  Honest Beauty Deep Hydration Face Cream with S...    Honest Beauty   
72  Hero Cosmetics Bright Eyes Illuminating Eye Cr...   Hero Cosmetics   
73  Palmers Cocoa Butter Formula Skin Therapy Oil ...         Palmer's   

      current_price            unit_price  \
0            $36.99        ($21.76/ounce)   
1   $11.79 - $26.99                   N/A   
2            $30.49  ($17.94/fluid ounce)   
3            $13.00                   N/A   
4            $33.49        ($19.70/ounce)   
..              ...                   ...   
69           $31.49                   N/A   
70           $14.59         ($8.58/ounce)   
71           $30.49  ($17.94/fluid ounce)   
72           $19.99                   N/A   
73           $14.69  ($14.69/fluid ounce)   

                             star_rating review_counts  is_on_sale  \
0    4.8 out of 5 stars with 757 ratings   757 reviews           0   
1   4.3 out of 5 stars with 3896 ratings  3896 reviews           0   
2    4.4 out of 5 stars with 236 ratings   236 reviews           0   
3   4.7 out of 5 stars with 6008 ratings  6008 reviews           0   
4   4.6 out of 5 stars with 5970 ratings  5970 reviews           0   
..                                   ...           ...         ...   
69  4.4 out of 5 stars with 1202 ratings  1202 reviews           0   
70  4.5 out of 5 stars with 1034 ratings  1034 reviews           0   
71   4.4 out of 5 stars with 420 ratings   420 reviews           0   
72   4.1 out of 5 stars with 124 ratings   124 reviews           0   
73   4.6 out of 5 stars with 701 ratings   701 reviews           0   

   is_free_shipping is_3day_delivery product_category  \
0                 1                1      Moisturizer   
1               N/A                1      Moisturizer   
2                 0                1      Moisturizer   
3                 0                1      Moisturizer   
4                 0                1      Moisturizer   
..              ...              ...              ...   
69                0                1      Moisturizer   
70                0                1      Moisturizer   
71                0                1      Moisturizer   
72                0                1      Moisturizer   
73                0                1      Moisturizer   

                                         product_link  
0   https://www.target.com/p/olay-super-facial-moi...  
1   https://www.target.com/p/la-roche-posay-toleri...  
2   https://www.target.com/p/good-clean-goop-the-n...  
3   https://www.target.com/p/e-l-f-holy-hydration-...  
4   https://www.target.com/p/l-39-oreal-paris-age-...  
..                                                ...  
69  https://www.target.com/p/roc-derm-correxion-du...  
70  https://www.target.com/p/cetaphil-rich-hydrati...  
71  https://www.target.com/p/honest-beauty-deep-hy...  
72  https://www.target.com/p/hero-cosmetics-bright...  
73  https://www.target.com/p/palmers-cocoa-butter-...  

[74 rows x 11 columns]

In [3]:
df.to_csv("moisturizer_data.csv",index=False)

# Page 4-6

In [4]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:124.0) Gecko/20100101 Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; rv:123.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (X11; openSUSE; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.140 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.139 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.170 Safari/537.36",
    "Mozilla/5.0 (X11; Linux Mint x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Manjaro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.193 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.151 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Kali) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.145 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Solus) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.95 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.148 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.120 Safari/537.36"
]



random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/moisturizers-skin-care-beauty/-/N-5xtzf?moveTo=product-list-grid&Nao=72"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list2 = []

def scrape_products():
    global product_list2, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list2.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Moisturizer",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list2)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：74 条


In [5]:
df2 = pd.DataFrame(product_list2)
df2

product_name          brand  \
0   L'Oreal Paris Revitalift Triple Power Fragranc...  L'Oreal Paris   
1   L'Oreal Paris Age Perfect Cell Renewal Midnigh...  L'Oreal Paris   
2   Honest Beauty Everyday Radiance + C Moisturize...  Honest Beauty   
3    PM Facial Moisturizing Lotion - 3 fl oz - up&up™          up&up   
4   Naturium The Smoother Glycolic Body Lotion - 8...       Naturium   
..                                                ...            ...   
69  Olay Age Defying 2-in-1 Anti-Wrinkle Day Cream...           Olay   
70            ELEMIS Pro-Collagen Marine Cream 3.3 oz         ELEMIS   
71  Current State Peptide + Caffeine Firming Eye C...  Current State   
72  Cetaphil Redness Relieving Night Face Moisturi...       Cetaphil   
73  TULA SKINCARE Instant De-Puff Eye Renewal Seru...  TULA SKINCARE   

   current_price            unit_price                           star_rating  \
0         $28.49        ($16.76/ounce)  4.6 out of 5 stars with 5038 ratings   
1         $33.49        ($19.70/ounce)  4.6 out of 5 stars with 5970 ratings   
2         $30.49  ($17.94/fluid ounce)   3.9 out of 5 stars with 215 ratings   
3         $13.49   ($4.50/fluid ounce)   4.5 out of 5 stars with 681 ratings   
4         $16.99         ($2.12/ounce)      4 out of 5 stars with 68 ratings   
..           ...                   ...                                   ...   
69        $14.94   ($8.79/fluid ounce)   4.5 out of 5 stars with 160 ratings   
70        $99.99                   N/A  4.8 out of 5 stars with 3158 ratings   
71        $16.49  ($32.98/fluid ounce)   4.4 out of 5 stars with 123 ratings   
72        $11.79   ($6.94/fluid ounce)   4.4 out of 5 stars with 302 ratings   
73        $58.00                   N/A   4.3 out of 5 stars with 771 ratings   

   review_counts  is_on_sale is_free_shipping is_3day_delivery  \
0   5038 reviews           0                0                1   
1   5970 reviews           0                0                1   
2    215 reviews           0                0                1   
3    681 reviews           0                0                1   
4     68 reviews           0                0                1   
..           ...         ...              ...              ...   
69   160 reviews           0                0                1   
70  3158 reviews           1                1                0   
71   123 reviews           0                0                1   
72   302 reviews           0                0                1   
73   771 reviews           0                1                1   

   product_category                                       product_link  
0       Moisturizer  https://www.target.com/p/l-39-oreal-paris-revi...  
1       Moisturizer  https://www.target.com/p/l-39-oreal-paris-age-...  
2       Moisturizer  https://www.target.com/p/honest-beauty-everyda...  
3       Moisturizer  https://www.target.com/p/pm-facial-moisturizin...  
4       Moisturizer  https://www.target.com/p/naturium-the-smoother...  
..              ...                                                ...  
69      Moisturizer  https://www.target.com/p/olay-age-defying-2-in...  
70      Moisturizer  https://www.target.com/p/elemis-pro-collagen-m...  
71      Moisturizer  https://www.target.com/p/current-state-peptide...  
72      Moisturizer  https://www.target.com/p/cetaphil-redness-reli...  
73      Moisturizer  https://www.target.com/p/tula-skincare-instant...  

[74 rows x 11 columns]

In [6]:
df2.to_csv("moisturizer_data2.csv",index=False)

# Page 7-9

In [9]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/moisturizers-skin-care-beauty/-/N-5xtzf?moveTo=product-list-grid&Nao=144"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list3 = []

def scrape_products():
    global product_list3, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list3.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Moisturizer",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list3)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：87 条


In [10]:
df3 = pd.DataFrame(product_list3)
df3

product_name         brand  \
0   RoC Retinol Correxion Line Smoothing Anti-Agin...           RoC   
1        cocokind Electrolyte Water Cream - 1.7 fl oz      cocokind   
2           ELEMIS Peptide4 Eye Recovery Cream 0.5 oz        ELEMIS   
3   Clinique For Men Oil-Free Moisturizer - 3.4 fl...      Clinique   
4   Origins A Perfect World Moisturizer - SPF 40 -...       Origins   
..                                                ...           ...   
82  Peach & Lily Pure Peach Retinoic Eye Cream - 0...  Peach & Lily   
83  Peach & Lily Matcha Pudding Antioxidant Cream ...  Peach & Lily   
84   Eminence Blueberry Soy Night Recovery Cream 2 oz      Eminence   
85  TRULY Hemp Oil Facial Serum - 1.7 fl oz - Ulta...         TRULY   
86  Clinique For Men Broad Spectrum SPF 21 Moistur...      Clinique   

   current_price            unit_price                           star_rating  \
0         $27.99                   N/A  4.5 out of 5 stars with 2434 ratings   
1         $24.49  ($14.41/fluid ounce)    4.3 out of 5 stars with 13 ratings   
2         $31.49                   N/A   4.5 out of 5 stars with 762 ratings   
3         $38.00  ($11.18/fluid ounce)    4.5 out of 5 stars with 53 ratings   
4         $58.00        ($34.12/ounce)  4.2 out of 5 stars with 1915 ratings   
..           ...                   ...                                   ...   
82        $43.00                   N/A   4.4 out of 5 stars with 593 ratings   
83        $43.00  ($25.44/fluid ounce)  4.5 out of 5 stars with 2266 ratings   
84        $48.99                   N/A       5 out of 5 stars with 2 ratings   
85        $28.00  ($16.47/fluid ounce)   4.5 out of 5 stars with 203 ratings   
86        $41.00  ($12.06/fluid ounce)   4.6 out of 5 stars with 118 ratings   

   review_counts  is_on_sale is_free_shipping  is_3day_delivery  \
0   2434 reviews           0                0                 1   
1     13 reviews           0                0                 1   
2    762 reviews           1                0                 0   
3     53 reviews           0                1                 1   
4   1915 reviews           0                1                 1   
..           ...         ...              ...               ...   
82   593 reviews           0                1                 0   
83  2266 reviews           0                1                 1   
84     2 reviews           1                1                 0   
85   203 reviews           0                0                 0   
86   118 reviews           0                1                 1   

   product_category                                       product_link  
0       Moisturizer  https://www.target.com/p/roc-retinol-correxion...  
1       Moisturizer  https://www.target.com/p/cocokind-electrolyte-...  
2       Moisturizer  https://www.target.com/p/elemis-peptide4-eye-r...  
3       Moisturizer  https://www.target.com/p/clinique-for-men-oil-...  
4       Moisturizer  https://www.target.com/p/origins-a-perfect-wor...  
..              ...                                                ...  
82      Moisturizer  https://www.target.com/p/peach-38-lily-pure-pe...  
83      Moisturizer  https://www.target.com/p/peach-38-lily-matcha-...  
84      Moisturizer  https://www.target.com/p/eminence-blueberry-so...  
85      Moisturizer  https://www.target.com/p/truly-hemp-oil-facial...  
86      Moisturizer  https://www.target.com/p/clinique-for-men-broa...  

[87 rows x 11 columns]

In [11]:
df3.to_csv("moisturizer_data3.csv",index=False)

# 汇总前9页

In [12]:
df_combined_moisturizer_1_9 = pd.concat([df,df3,df3], axis=0, ignore_index=True)
df_combined_moisturizer_1_9

product_name            brand  \
0       Olay Super Facial Moisturizer - SPF 30 - 1.7oz             Olay   
1    La Roche Posay Toleriane Double Repair Face Mo...   La Roche Posay   
2    good.clean.goop beauty The Nutrient-Rich Daily...  good.clean.goop   
3    e.l.f. Holy Hydration Face Cream Fragrance Fre...           e.l.f.   
4    L'Oreal Paris Age Perfect Cell Renewal Midnigh...    L'Oreal Paris   
..                                                 ...              ...   
243  Peach & Lily Pure Peach Retinoic Eye Cream - 0...     Peach & Lily   
244  Peach & Lily Matcha Pudding Antioxidant Cream ...     Peach & Lily   
245   Eminence Blueberry Soy Night Recovery Cream 2 oz         Eminence   
246  TRULY Hemp Oil Facial Serum - 1.7 fl oz - Ulta...            TRULY   
247  Clinique For Men Broad Spectrum SPF 21 Moistur...         Clinique   

       current_price            unit_price  \
0             $36.99        ($21.76/ounce)   
1    $11.79 - $26.99                   N/A   
2             $30.49  ($17.94/fluid ounce)   
3             $13.00                   N/A   
4             $33.49        ($19.70/ounce)   
..               ...                   ...   
243           $43.00                   N/A   
244           $43.00  ($25.44/fluid ounce)   
245           $48.99                   N/A   
246           $28.00  ($16.47/fluid ounce)   
247           $41.00  ($12.06/fluid ounce)   

                              star_rating review_counts  is_on_sale  \
0     4.8 out of 5 stars with 757 ratings   757 reviews           0   
1    4.3 out of 5 stars with 3896 ratings  3896 reviews           0   
2     4.4 out of 5 stars with 236 ratings   236 reviews           0   
3    4.7 out of 5 stars with 6008 ratings  6008 reviews           0   
4    4.6 out of 5 stars with 5970 ratings  5970 reviews           0   
..                                    ...           ...         ...   
243   4.4 out of 5 stars with 593 ratings   593 reviews           0   
244  4.5 out of 5 stars with 2266 ratings  2266 reviews           0   
245       5 out of 5 stars with 2 ratings     2 reviews           1   
246   4.5 out of 5 stars with 203 ratings   203 reviews           0   
247   4.6 out of 5 stars with 118 ratings   118 reviews           0   

    is_free_shipping is_3day_delivery product_category  \
0                  1                1      Moisturizer   
1                N/A                1      Moisturizer   
2                  0                1      Moisturizer   
3                  0                1      Moisturizer   
4                  0                1      Moisturizer   
..               ...              ...              ...   
243                1                0      Moisturizer   
244                1                1      Moisturizer   
245                1                0      Moisturizer   
246                0                0      Moisturizer   
247                1                1      Moisturizer   

                                          product_link  
0    https://www.target.com/p/olay-super-facial-moi...  
1    https://www.target.com/p/la-roche-posay-toleri...  
2    https://www.target.com/p/good-clean-goop-the-n...  
3    https://www.target.com/p/e-l-f-holy-hydration-...  
4    https://www.target.com/p/l-39-oreal-paris-age-...  
..                                                 ...  
243  https://www.target.com/p/peach-38-lily-pure-pe...  
244  https://www.target.com/p/peach-38-lily-matcha-...  
245  https://www.target.com/p/eminence-blueberry-so...  
246  https://www.target.com/p/truly-hemp-oil-facial...  
247  https://www.target.com/p/clinique-for-men-broa...  

[248 rows x 11 columns]

In [13]:
df_combined_moisturizer_1_9.to_csv("moisturizer_data_1_9.csv",index=False)